In [ ]:
import galsim
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from chromatic_weak_lensing.diffsky import Diffsky, RomanRubin

In [ ]:
# ! curl https://portal.nersc.gov/project/hacc/aphearin/lsstdesc_diffsky_data/roman_rubin_2023_z_0_1_cutout_9043.testdata.hdf5 > diffsky.testdata.hdf5

In [ ]:
# input data is structured like a dictionary (columnar access)

data = {}

fn = "diffsky.testdata.hdf5"
with h5py.File(fn) as hf:
    for key in hf.keys():
        if key != "metaData":
            snap = hf[key]
            for field in snap.keys():
                if field not in data:
                    data[field] = np.array([])
                data[field] = np.append(data[field], snap[field][:])
        else:
            for k, v in hf[key].items():
                print(f"{k}: {v[()]}")

In [ ]:
# A hdf5 group will also work

# hf = h5py.File(fn)
# data = hf["247"]

In [ ]:
# A pyarrow data source can also be used

# import pyarrow.dataset as ds

# dataset = ds.dataset("roman_rubin_2023_v1.1.3_parquet")
# data = dataset.head(100, columns=roman_rubin.columns, filter=(ds.field("LSST_obs_i") < 26))

In [ ]:
roman_rubin = RomanRubin(data)
diffsky = Diffsky()

In [ ]:
fig, axs = plt.subplots(1, 1)

axs.set_title("Diffsky Galaxy Morphology")

i = 0

params = roman_rubin.get_morphology_params(i)
morph = diffsky.get_morphology(*params)

psf = galsim.Gaussian(fwhm=0.7)
observed = galsim.Convolve([psf, morph])
image = observed.drawImage(scale=0.2)

axs.imshow(image.array, origin="lower")

plt.show()

In [ ]:
fig, axs = plt.subplots(1, 1)

axs.set_yscale("log")
axs.set_xlabel("$\lambda$ [$nm$]")
axs.set_ylabel("$f_{photons}$ [$photons/nm/cm^2/s$]")
axs.set_title("Diffsky Galactic Spectra")

wl = np.linspace(300, 1200, 1000)

for i in range(len(roman_rubin.data["redshift"])):
    params = roman_rubin.get_spectrum_params(i)
    spec = diffsky.get_spectrum(*params)

    axs.plot(wl, spec(wl))

plt.show()

In [ ]:
fig, axs = plt.subplot_mosaic(
    [
        ["g", "r", "i"],
        ["sed", "sed", "sed"]
    ],
    constrained_layout=True,
)

axs["sed"].set_yscale("log")
axs["sed"].set_xlabel("$\lambda$ [$nm$]")
axs["sed"].set_ylabel("$f_{photons}$ [$photons/nm/cm^2/s$]")

axs["g"].set_title("g")
axs["r"].set_title("r")
axs["i"].set_title("i")

fig.suptitle("Diffsky Galaxy")

wl = np.linspace(300, 1200, 1000)

i = -1

params = roman_rubin.get_params(i)
galaxy = diffsky.get_galaxy(*params)

base_psf = galsim.Gaussian(fwhm=0.7)
psf = galsim.ChromaticAtmosphere(
    base_psf,
    700,
    alpha=-0.3,
    zenith_angle=45 * galsim.degrees,
    parallactic_angle=0 * galsim.degrees,
)

observed = galsim.Convolve([psf, galaxy])
g_image = observed.drawImage(nx=53, ny=53, scale=0.2, bandpass=galsim.Bandpass("LSST_g.dat", "nm"))
r_image = observed.drawImage(nx=53, ny=53, scale=0.2, bandpass=galsim.Bandpass("LSST_r.dat", "nm"))
i_image = observed.drawImage(nx=53, ny=53, scale=0.2, bandpass=galsim.Bandpass("LSST_i.dat", "nm"))

norm = mpl.colors.Normalize()

axs["sed"].plot(wl, galaxy.sed(wl))
axs["g"].imshow(g_image.array, norm=norm, origin="lower")
axs["r"].imshow(r_image.array, norm=norm, origin="lower")
axs["i"].imshow(i_image.array, norm=norm, origin="lower")

plt.show()
